In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import gc
from tqdm import tqdm

In [2]:
import psycopg2

connection = psycopg2.connect(database="mimiciv", user="uti_user", host='127.0.0.1', port=5432, password='s0f4C1#4')

cursor = connection.cursor()

DETAIL:  The database was created using collation version 2.35, but the operating system provides version 2.39.
HINT:  Rebuild all objects in this database that use the default collation and run ALTER DATABASE mimiciv REFRESH COLLATION VERSION, or build PostgreSQL with the right library version.


In [87]:
def get_data(query,med_type):
    cursor.execute(query)
    record = cursor.fetchall()
    # Get column names from cursor description
    columns = [desc[0] for desc in cursor.description]

    # Convert to DataFrame
    df = pd.DataFrame(record, columns=columns)
    binary = df[['subject_id','hadm_id','stay_id']]
    binary[f'use_{med_type}'] = df['use']
    
    count = df[['subject_id','hadm_id','stay_id']]
    count[f'qtd_{med_type}'] = df['qtd']
    return binary,count
    

In [ ]:
query = '''WITH abx AS (
    SELECT DISTINCT
        drug
        , route
        , CASE
            WHEN LOWER(drug) LIKE  '%adeno%' THEN 1
            WHEN LOWER(drug) LIKE  '%cilos%' THEN 1
            WHEN LOWER(drug) LIKE  '%diazoxide%' THEN 1
            WHEN LOWER(drug) LIKE  '%milrinone%' THEN 1
            WHEN LOWER(drug) LIKE  '%minoxidil%' THEN 1
            WHEN LOWER(drug) LIKE  '%papaverine%' THEN 1
            WHEN LOWER(drug) LIKE  '%pentoxifylline%' THEN 1
            WHEN LOWER(drug) LIKE  '%prazosin%' THEN 1
            WHEN LOWER(drug) LIKE  '%yohimbine%' THEN 1
            WHEN LOWER(drug) LIKE  '%inositol%' THEN 1
            WHEN LOWER(drug) LIKE  '%nitroprusside%' THEN 1
            ELSE 0
        END AS antibiotic
    FROM mimiciv_hosp.prescriptions
    -- excludes vials/syringe/normal saline, etc
    WHERE drug_type NOT IN ('BASE')
        -- we exclude routes via the eye, ears, or topically
        AND route NOT IN ('OU', 'OS', 'OD', 'AU', 'AS', 'AD', 'TP')
        AND LOWER(route) NOT LIKE '%ear%'
        AND LOWER(route) NOT LIKE '%eye%'
        -- we exclude certain types of antibiotics: topical creams,
        -- gels, desens, etc
        AND LOWER(drug) NOT LIKE '%cream%'
        AND LOWER(drug) NOT LIKE '%desensitization%'
        AND LOWER(drug) NOT LIKE '%ophth oint%'
        AND LOWER(drug) NOT LIKE '%gel%'
-- other routes not sure about...
-- for sure keep: ('IV','PO','PO/NG','ORAL', 'IV DRIP', 'IV BOLUS')
-- ? VT, PB, PR, PL, NS, NG, NEB, NAS, LOCK, J TUBE, IVT
-- ? IT, IRR, IP, IO, INHALATION, IN, IM
-- ? IJ, IH, G TUBE, DIALYS
-- ?? enemas??
)

SELECT  
    sp.subject_id,  
    ie.hadm_id,  
    sp.stay_id,  
    CASE  
        WHEN SUM(CASE WHEN pr.starttime < ie.intime + INTERVAL '1 day' THEN abx.antibiotic ELSE 0 END) >= 1  
        THEN 1  
        ELSE 0  
    END AS use,  
    SUM(CASE WHEN pr.starttime < ie.intime + INTERVAL '1 day' THEN abx.antibiotic ELSE 0 END) AS qtd
FROM mimiciv_derived.sepsis3 sp  
LEFT JOIN mimiciv_icu.icustays ie  
    ON ie.stay_id = sp.stay_id  
LEFT JOIN mimiciv_hosp.prescriptions pr  
    ON pr.hadm_id = ie.hadm_id  
    AND pr.starttime >= ie.intime  
    AND pr.starttime < ie.outtime  
LEFT JOIN abx  
    ON pr.drug = abx.drug  
    AND pr.route = abx.route  
GROUP BY sp.stay_id, ie.hadm_id, sp.subject_id;
    
; '''
binary_vasodilators,count_vasodilators = get_data(query,'vasodilators')



/tmp/ipykernel_114470/460538831.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  binary[f'use_{med_type}'] = df['use']
/tmp/ipykernel_114470/460538831.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  count[f'qtd_{med_type}'] = df['qtd']


In [ ]:
binary_anticoagulants
count_anticoagulants
binary_antibiotics
count_antibiotics
binary_antiplatelets
count_antiplatelets
binary_antipsychotics
count_antipsychotics
binary_curares
count_curares
binary_inhibitors
count_inhibitors
binary_painkillers_sedatives
count_painkillers_sedatives
binary_vasoagents
count_vasoagents
binary_vasodilators
count_vasodilators

,subject_id,hadm_id,stay_id,qtd_antipsychotics
0,18581793,29083316,33874185,0
1,18360993,29321439,35561103,2
2,15711193,24898735,33921216,0
3,15407803,22894295,36111321,1
4,10516941,29234231,36734224,0
...,...,...,...,...
32966,16622839,25514759,36110673,0
32967,14045654,24160410,31290077,0
32968,12570801,24828158,33163467,2
32969,18781367,24995300,31520661,0


In [ ]:
categories = [
    "anticoagulants", "antibiotics", "antiplatelets", "antipsychotics", "curares",
    "inhibitors", "painkillers_sedatives", "vasoagents", "vasodilators"
]

# Inicializa os DataFrames finais
medications_count = None
medications_binary = None

# Une todos os DataFrames count
for category in categories:
    df_count = globals()[f"count_{category}"]
    
    if medications_count is None:
        medications_count = df_count
    else:
        medications_count = medications_count.merge(df_count, on=["stay_id", "subject_id", "hadm_id"], how="outer")

# Une todos os DataFrames binary
for category in categories:
    df_binary = globals()[f"binary_{category}"]
    
    if medications_binary is None:
        medications_binary = df_binary
    else:
        medications_binary = medications_binary.merge(df_binary, on=["stay_id", "subject_id", "hadm_id"], how="outer")

# Exibir os DataFrames resultantes
medications_count.head()
medications_binary.head()


,subject_id,hadm_id,stay_id,use_anticoagulants,use_antibiotics,use_antiplatelets,use_antipsychotics,use_curares,use_inhibitors,use_painkillers_sedatives,use_vasoagents,use_vasodilators
0,18421337,22413411,30000484,1,1,1,0,0,0,1,1,0
1,12207593,22795209,30000646,1,1,0,0,0,0,0,1,0
2,16513856,24463832,30001446,0,1,0,0,0,1,0,1,0
3,10656173,25778760,30001555,1,1,0,0,0,0,0,0,0
4,17921898,28841024,30002415,0,1,1,0,0,0,1,1,0


In [110]:
medications_binary.to_csv('/scratch/haniel.botelho/physionet.org/files/mimiciv/2.2/Data24h/medications_binary.csv',index=False)